## 0. Setup

In [52]:
# Import dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import random
import numpy as np

In [53]:
# Set up your device 
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

In [54]:
# Set up random seed to 1008. Do not change the random seed.
# Yes, these are all necessary when you run experiments!
seed = 1008
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

## 1. Data: MNIST
#### Load the MNIST training and test dataset using $\texttt{torch.utils.data.DataLoader}$ and $\texttt{torchvision.datasets}$. 

Hint: You might find Alf's notebook useful: https://github.com/Atcold/pytorch-Deep-Learning/blob/master/06-convnet.ipynb, or see some of the PyTorch tutorials.

### 1.1. Load Training Set [4 pts]

In [55]:
# Load the MNIST training set with batch size 128, apply data shuffling and normalization

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=128, shuffle=True)

### 1.1. Load Test Set [4 pts]

In [56]:
# Load the MNIST test set with batch size 128, apply data shuffling and normalization
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=128, shuffle=True)

## 2. Models
#### You are going to define two convolutional neural networks which are trained to classify MNIST digits

### 2.1. CNN without Batch Norm [5 pts]

In [57]:
# Fill in the values below that make this network valid for MNIST data

conv1_in_ch = 1
conv2_in_ch = 20
fc1_in_features = 800
fc2_in_features = 500
n_classes = 10

In [58]:
class NetWithoutBatchNorm(nn.Module):
    def __init__(self):
        super(NetWithoutBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=conv1_in_ch, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=conv2_in_ch, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=500)
        self.fc2 = nn.Linear(in_features=fc2_in_features, out_features=n_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, fc1_in_features) # reshaping
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # Return the log_softmax of x.
        return F.log_softmax(x,dim=1)

### 2.2. CNN with Batch Norm [5 pts]

In [59]:
# Fill in the values below that make this network valid for MNIST data

conv1_bn_size = conv2_in_ch
conv2_bn_size = 50
fc1_bn_size = fc2_in_features

In [60]:
# Define the CNN with architecture explained in Part 2.2
class NetWithBatchNorm(nn.Module):
    def __init__(self):
        super(NetWithBatchNorm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=conv1_in_ch, out_channels=20, kernel_size=5, stride=1)
        self.conv1_bn = nn.BatchNorm2d(conv1_bn_size)
        self.conv2 = nn.Conv2d(in_channels=conv2_in_ch, out_channels=50, kernel_size=5, stride=1)
        self.conv2_bn = nn.BatchNorm2d(conv2_bn_size)
        self.fc1 = nn.Linear(in_features=fc1_in_features, out_features=500)
        self.fc1_bn = nn.BatchNorm1d(fc1_bn_size)
        self.fc2 = nn.Linear(in_features=fc2_in_features, out_features=n_classes)

    def forward(self, x):
        x = F.relu(self.conv1_bn(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, fc1_in_features)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
        # Return the log_softmax of x.
        return F.log_softmax(x,dim=1)

## 3. Training & Evaluation

### 3.1. Define training method [10 pts]

In [61]:
def train(model, device, train_loader, optimizer, epoch, log_interval = 100):
    # Set model to training mode
    model.train()
    # Loop through data points
    for batch_idx, (data, target) in enumerate(train_loader):
    
        # Send data and target to device
        # TODO
        data = data.to(device=device)
        target = target.to(device=device,dtype=torch.long)
        
        # Zero out the ortimizer
        optimizer.zero_grad()
        
        # Pass data through model
        output = model(data)
        
        # Compute the negative log likelihood loss
        loss = F.nll_loss(output,target)
        
        # Backpropagate loss
        loss.backward()
        
        # Make a step with the optimizer
        optimizer.step()
        
        # Print loss (uncomment lines below once implemented)
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

### 3.2. Define test method [10 pts]

In [66]:
# Define test method
def test(model, device, test_loader):
    # Set model to evaluation mode
    model.eval()
    # Variable for the total loss 
    test_loss = 0
    # Counter for the correct predictions
    num_correct = 0
    
    # don't need autograd for eval
    with torch.no_grad():
        # Loop through data points
        for data, target in test_loader:
        
            # Send data to device
            data = data.to(device=device)
            target = target.to(device=device, dtype=torch.long)
            
            # Pass data through model
            output = model(data)
            
            # Compute the negative log likelihood loss with reduction='sum' and add to total test_loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            
            # Get predictions from the model for each data point
            pred = output.data.max(1,keepdim=True)[1]
            
            # Add number of correct predictions to total num_correct 
            num_correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
    
    # Compute the average test_loss
    avg_test_loss = test_loss / len(test_loader.dataset)
    
    # Print loss (uncomment lines below once implemented)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        avg_test_loss, num_correct, len(test_loader.dataset),
        100. * num_correct / len(test_loader.dataset)))

### 3.3 Train NetWithoutBatchNorm() [5 pts]

In [67]:
# Deifne model and sent to device
model = NetWithoutBatchNorm()
model.to(device)

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(),lr =0.02, momentum= 0.5)

# Training loop with 10 epochs
for epoch in range(1, 10 + 1):

    # Train model
    train(model,device,train_loader,optimizer,epoch)
    
    # Test model
    test(model,device,test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.291516
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.231629
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.144679
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.112140
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.096578

Test set: Average loss: 0.0920, Accuracy: 9715/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.121055
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.186163
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.100891
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.057935
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.118874

Test set: Average loss: 0.0668, Accuracy: 9778/10000 (98%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.097909
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.071000
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.013582
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.035494
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.014610

Test set: Average loss: 0.0573, Accuracy: 9831/10000 (98%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.036453
Train Epoch: 4 [12800/60000 (21%)]	Lo

### 3.4 Train NetWithBatchNorm() [5 pts]

In [71]:
# Define model and sent to device
model = NetWithBatchNorm()
model.to(device)

# Optimizer: SGD with learning rate of 1e-2 and momentum of 0.5
optimizer = optim.SGD(model.parameters(),lr=0.02,momentum=0.5)

# Training loop with 10 epochs
for epoch in range(1, 10 + 1):
    
    # Train model
    train(model, device, train_loader, optimizer, epoch)
    
    # Test model
    test(model,device,test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.403052
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.122699
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.199911
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.135071
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.085357

Test set: Average loss: 0.0538, Accuracy: 9859/10000 (99%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.091993
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.070561
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.070740
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.027198
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.036871

Test set: Average loss: 0.0398, Accuracy: 9879/10000 (99%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.031396
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.012027
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.042147
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.020955
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.028266

Test set: Average loss: 0.0317, Accuracy: 9908/10000 (99%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.028212
Train Epoch: 4 [12800/60000 (21%)]	Lo

## 4. Empirically, which of the models achieves higher accuracy faster? [2 pts]

Answer: NetwithBachNorm reaches higher accuracy faster. 